In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os as os
import math

#TODO Introducir el dataset con el que se vaya a trabajar
station = 'palmer'
df = pd.read_csv(f'./data/data_{station}/{station}_raw.csv')
cols = [col for col in df.columns]

In [2]:
df.head()

,date,vel,dir,gust_vel,gust_dir,temp,hr,dew,pres,snow_depth,prec
0,2015-12-01 00:00:00,7.47,270.0,9.57,287.0,-1.3,83.0,-3.9,978.3,NaN,0.0
1,2015-12-01 00:01:00,7.10,270.0,8.90,281.0,-1.3,82.0,-3.9,978.3,NaN,0.0
2,2015-12-01 00:02:00,7.38,278.0,8.85,285.0,-1.3,82.0,-4.0,978.3,NaN,0.0
3,2015-12-01 00:03:00,7.48,278.0,8.85,285.0,-1.3,82.0,-4.0,978.3,NaN,0.0
4,2015-12-01 00:04:00,7.02,280.0,8.85,285.0,-1.3,82.0,-4.0,978.3,NaN,0.0


In [3]:

# Pasamos la columna 'date' a formato datetime

try:
    print('ok')
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date')
except ValueError:  # Capturamos la excepción correcta
    for i in range(len(df)):
        fecha = df.loc[i, 'date']  

        if len(fecha) == 10:  # Verificamos si la fecha no tiene hora
            fecha += ' 00:00:00'  # Añadimos '00:00:00'
        
            # Convertimos la cadena a formato datetime
            df.loc[i, 'date'] = pd.to_datetime(fecha)


df.info()

ok
<class 'pandas.core.frame.DataFrame'>
Index: 4168070 entries, 86375 to 4079816
Data columns (total 11 columns):
 #   Column      Dtype         
---  ------      -----         
 0   date        datetime64[ns]
 1   vel         float64       
 2   dir         float64       
 3   gust_vel    float64       
 4   gust_dir    float64       
 5   temp        float64       
 6   hr          float64       
 7   dew         float64       
 8   pres        float64       
 9   snow_depth  float64       
 10  prec        float64       
dtypes: datetime64[ns](1), float64(10)
memory usage: 381.6 MB


In [4]:
df.head()


,date,vel,dir,gust_vel,gust_dir,temp,hr,dew,pres,snow_depth,prec
86375,2015-10-01 00:00:00,8.04,293.0,9.67,295.0,-2.4,87.0,-4.3,980.9,NaN,0.0
86376,2015-10-01 00:01:00,7.78,293.0,9.67,295.0,-2.4,87.0,-4.3,980.9,NaN,0.0
86377,2015-10-01 00:02:00,7.49,291.0,9.67,295.0,-2.4,87.0,-4.3,980.9,NaN,0.0
86378,2015-10-01 00:03:00,7.43,291.0,9.62,290.0,-2.5,87.0,-4.3,981.0,NaN,0.0
86379,2015-10-01 00:04:00,6.66,293.0,9.62,290.0,-2.5,87.0,-4.3,981.0,NaN,0.0


In [12]:
df['date'] = pd.to_datetime(df['date'])


In [5]:

# Filtrar las filas con fechas inválidas
filas_invalidas = df[df['date'].isna()]

# # Mostrar las filas con fechas incorrectas
# print(filas_invalidas)

num_filas_invalidas = filas_invalidas.isnull().sum()
num_filas_invalidas


date          0
vel           0
dir           0
gust_vel      0
gust_dir      0
temp          0
hr            0
dew           0
pres          0
snow_depth    0
prec          0
dtype: int64

In [19]:
df = df[(df['date'].dt.year >= 2007) & (df['date'].dt.month >= 2) & (df['date'].dt.day <= 10)]


In [6]:
df.head()

,date,vel,dir,gust_vel,gust_dir,temp,hr,dew,pres,snow_depth,prec
86375,2015-10-01 00:00:00,8.04,293.0,9.67,295.0,-2.4,87.0,-4.3,980.9,NaN,0.0
86376,2015-10-01 00:01:00,7.78,293.0,9.67,295.0,-2.4,87.0,-4.3,980.9,NaN,0.0
86377,2015-10-01 00:02:00,7.49,291.0,9.67,295.0,-2.4,87.0,-4.3,980.9,NaN,0.0
86378,2015-10-01 00:03:00,7.43,291.0,9.62,290.0,-2.5,87.0,-4.3,981.0,NaN,0.0
86379,2015-10-01 00:04:00,6.66,293.0,9.62,290.0,-2.5,87.0,-4.3,981.0,NaN,0.0


In [3]:

# En caso de que los nulos tengan un formato diferente

df.replace(444, np.nan, inplace=True)

### Calcular la humedad relativa

Se aplica la formula de magnus a partir de la temperatura y el punto de rocío.

In [14]:
# Definir la función para calcular la presión de vapor de saturación utilizando la ecuación de Magnus
def presion_vapor_saturacion(T):
    return 6.1094 * np.exp((17.625 * T) / (T + 243.04))

# Definir la función para calcular la humedad relativa
def calcular_humedad_relativa(temp_aire, temp_rocio):
    # Calcular la presión de vapor para la temperatura del aire y el punto de rocío
    e_aire = presion_vapor_saturacion(temp_aire)
    e_rocio = presion_vapor_saturacion(temp_rocio)
    
    # Calcular la humedad relativa
    humedad_relativa = 100 * (e_rocio / e_aire)
    
    return humedad_relativa

# Aplicar la función a tu DataFrame, asumiendo que 'temp' es la temperatura del aire y 'd2m' el punto de rocío
df['hr'] = calcular_humedad_relativa(df['temp'], df['d2m'])

# Mostrar las primeras filas con la nueva columna de humedad relativa
print(df[['temp', 'd2m', 'hr']].head(30))

        temp  d2m          hr
201207   1.9  1.9  100.000000
201206   1.5  1.5  100.000000
201205   2.0  2.0  100.000000
201204   1.6  1.6  100.000000
201203   5.5  2.9   83.343240
201202   5.4  2.0   78.722874
201201   3.7  1.5   85.540069
201200   3.6  1.6   86.763330
201199   3.4  1.5   87.367093
201198   3.5  1.6   87.376700
201197   3.5  1.6   87.376700
201196   3.8  1.6   85.550941
201195   3.5  1.5   86.753294
201194   3.7  1.5   85.540069
201193   3.1  1.6   89.878996
201192   2.7  1.9   94.465770
201191   2.8  1.6   91.808170
201190   3.4  1.5   87.367093
201189   3.7  1.4   84.929269
201188   2.4  1.8   95.813257
201187   2.2  1.9   97.883395
201186   2.5  2.0   96.502879
201185   2.7  2.1   95.823269
201184   2.7  1.6   92.461451
201183   2.2  1.5   95.123011
201182   1.0  1.0  100.000000
201181   1.2  1.2  100.000000
201180   0.8  0.8  100.000000
201179   1.3  1.3  100.000000
201178   0.5  0.5  100.000000


In [7]:
cols

['date',
 'vel',
 'dir',
 'gust_vel',
 'gust_dir',
 'temp',
 'hr',
 'dew',
 'pres',
 'snow_depth',
 'prec']

### Uniformamos el dataset

-Eliminamos las columnas que no nos interesen

-Fijamos la columan de fecha

In [8]:
# date,temp,skt,dir,vel,hr,prec,pres

columns_to_drop = [
 'gust_vel',
 'gust_dir',
  'dew', 'snow_depth'
    ]

df_2 = df.drop(columns=columns_to_drop)

df_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4168070 entries, 86375 to 4079816
Data columns (total 7 columns):
 #   Column  Dtype         
---  ------  -----         
 0   date    datetime64[ns]
 1   vel     float64       
 2   dir     float64       
 3   temp    float64       
 4   hr      float64       
 5   pres    float64       
 6   prec    float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 254.4 MB


In [23]:
df_2['date'] = df_2['date'].apply(lambda x: x if ':' in str(x) else str(x) + ' 00:00:00')
df_2['date'].head()

df_2['date'] = pd.to_datetime(df_2['date'])

In [9]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4168070 entries, 86375 to 4079816
Data columns (total 7 columns):
 #   Column  Dtype         
---  ------  -----         
 0   date    datetime64[ns]
 1   vel     float64       
 2   dir     float64       
 3   temp    float64       
 4   hr      float64       
 5   pres    float64       
 6   prec    float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 254.4 MB


In [10]:
#Recuento de nulos

df_2.head()

,date,vel,dir,temp,hr,pres,prec
86375,2015-10-01 00:00:00,8.04,293.0,-2.4,87.0,980.9,0.0
86376,2015-10-01 00:01:00,7.78,293.0,-2.4,87.0,980.9,0.0
86377,2015-10-01 00:02:00,7.49,291.0,-2.4,87.0,980.9,0.0
86378,2015-10-01 00:03:00,7.43,291.0,-2.5,87.0,981.0,0.0
86379,2015-10-01 00:04:00,6.66,293.0,-2.5,87.0,981.0,0.0


In [18]:
# Recuento de fechas duplicadas
duplicates = df_2.duplicated(subset='date', keep='first').sum()

duplicates

0

### Reindexación de las columnas

In [12]:
cols = [col for col in df_2.columns]
cols

['date', 'vel', 'dir', 'temp', 'hr', 'pres', 'prec']

In [13]:
# date,temp,skt,dir,vel,hr,prec,pres
new_order = ['date', 'temp', 'dir', 'vel', 'hr', 'prec' , 'pres']
df_2 = df_2.reindex(columns=new_order)
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4168070 entries, 86375 to 4079816
Data columns (total 7 columns):
 #   Column  Dtype         
---  ------  -----         
 0   date    datetime64[ns]
 1   temp    float64       
 2   dir     float64       
 3   vel     float64       
 4   hr      float64       
 5   prec    float64       
 6   pres    float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 254.4 MB


In [14]:
df_2.to_csv(f'./data/data_{station}/{station}_clean.csv', index=False, columns=new_order)

# Visualizacion de los datos

In [81]:
station = 'gdc'

In [82]:
df = pd.read_csv(f'./data/data_{station}/{station}_clean.csv')

In [9]:

num_cols = len(df_2.columns) - 1  
num_rows = math.ceil(num_cols / 4)  


fig, axes = plt.subplots(nrows=num_rows, ncols=4, figsize=(20, 5 * num_rows))  

# Aplanar el array de ejes para poder iterar sobre él
axes = axes.flatten()

# Iterar sobre las columnas (excluyendo 'fhora') y generar gráficos
for ax, col in zip(axes, df_2.columns.drop('date')):
    print(col, '--->' ,type(df_2[col]))
    df_2[col] = pd.to_numeric(df_2[col], errors='coerce')
    ax.plot(df_2['date'], df_2[col])
    ax.set_xlabel('date')
    ax.set_ylabel(col)
    ax.set_title(f'{col} a lo largo del tiempo')

# Eliminar los ejes sobrantes si el número de columnas no es múltiplo de 4
if num_cols % 4:
    for ax in axes[num_cols:]:
        ax.remove()

plt.savefig(f'resource/clean_{station}.png')

plt.tight_layout()
plt.show()

skt ---> <class 'pandas.core.series.Series'>
dir ---> <class 'pandas.core.series.Series'>
vel ---> <class 'pandas.core.series.Series'>
hr ---> <class 'pandas.core.series.Series'>
prec ---> <class 'pandas.core.series.Series'>
pres ---> <class 'pandas.core.series.Series'>


KeyboardInterrupt: 